# Import necessary libraries

In [1]:
!pip install finnhub-python
import yfinance as yf
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta, date
import json


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


# Import Data

In [2]:
current_directory = os.getcwd()  # Get the current working directory
print(current_directory)  # Outputs the current directory
subdirectory = 'manual_inputs'  # This is your target subfolder
file_name = 'all_tickers.txt'  # Name of the file to save
file_path = os.path.join(current_directory, subdirectory, file_name)  # Full path for the file

with open(file_path, 'r') as file:
    all_tickers = file.read().splitlines()

# with open('all_tickers.txt', 'r') as file:
#     all_tickers = file.read().splitlines()
    
# Market Indices
tickers = [
    '^IXIC',
    '^VIX',  # CBOE Volatility Index
    '^TNX',  # 10-Year Treasury Note Yield
    '^IRX',  # 13-Week Treasury Bill Yield
]

# Bonds and Fixed Income ETFs
tickers += [
    'TLT',   # iShares 20+ Year Treasury Bond ETF
    'TMF',   # Direxion Daily 20-Year Treasury Bull 3X Shares
    'SHY',   # iShares 1-3 Year Treasury Bond ETF
    'EDV',   # Vanguard Extended Duration Treasury ETF
    'ZROZ',  # PIMCO 25+ Year Zero Coupon U.S. Treasury Index ETF
    'GOVT',  # Vanguard U.S. Government Bond ETF
]

# Volatility and Inverse ETFs
tickers += [
    'VXX',   # iPath Series B S&P 500 VIX Short-Term Futures ETN
    'UVXY',  # ProShares Ultra VIX Short-Term Futures ETF
]

# Growth and Value ETFs
tickers += [
    'MTUM',  # iShares Edge MSCI USA Momentum Factor ETF
    'SPY',   # SPDR S&P 500 ETF Trust
    'QQQ',   # Invesco QQQ Trust (Nasdaq 100)
    'QQQM',  # Invesco NASDAQ 100 ETF (mini)
    'VOOG',  # Vanguard S&P 500 Growth ETF
    'VGT',   # Vanguard Information Technology ETF
    'XLU',   # Utilities Select Sector SPDR Fund
]

# Large-Cap Stocks
tickers += [
    'GOOG',  # Alphabet Inc. (Class C)
    'AVGO',  # Broadcom Inc.
    'AMZN',  # Amazon.com, Inc.
    'NVDA',  # NVIDIA Corporation
    'MSFT',  # Microsoft Corporation
    'DIS',   # The Walt Disney Company
    'AAPL',  # Apple Inc.
    'WMT',   # Walmart Inc.
    'META',  # Meta Platforms, Inc. (formerly Facebook)
    'COST',  # Costco Wholesale Corporation
]

# Precious Metals
tickers += [
    'GLD',   # SPDR Gold Shares
    'IAU',   # iShares Gold Trust
    'PHYS',  # Sprott Physical Gold Trust
    'GDX',   # VanEck Vectors Gold Miners ETF
    'SLV',   # iShares Silver Trust
    'IAU',   # iShares Gold Trust (duplicate)
]

# Small and Mid-Cap Stocks / Growth Companies
tickers += [
    'SFM',   # Sprouts Farmers Market, Inc.
    'PLTR',  # Palantir Technologies Inc.
    'QUBT',  # Quantum Computing Inc.
    'IONQ',  # IonQ, Inc. (quantum computing)
    'APLD',  # Applied Digital Corporation
    'LMND',  # Lemonade, Inc.
    'VRT',   # Vertiv Holdings Co.
    'RKLB',  # Rocket Lab USA, Inc.
    'AFRM',  # Affirm Holdings, Inc.
    'FLR',   # Fluor Corporation
    'ESOA',  # Eos Energy Enterprises, Inc.
    'BKNG',  # Booking Holdings Inc.
    'HQY',   # HealthEquity, Inc.
    'NU',    # Nu Holdings Ltd.
    'VNO',   # Vornado Realty Trust
    'MP',    # MP Materials Corp.
    'SMR',   # SM Energy Company
    'IDCC',  # InterDigital, Inc.
    'RVMD',  # Reviva Pharmaceuticals Holdings, Inc.
    'DHI',   # D.R. Horton, Inc.
    'PANW',  # Palo Alto Networks, Inc.
    'DASH',  # DoorDash, Inc.
    'CHWY',  # Chewy, Inc.
    'WPM',   # Wheaton Precious Metals Corp.
    'SE',    # Sea Limited
     'T',    # AT&T Inc.
    'LMT',   # Lockheed Martin Corporation
    'TKO',   # Take-Two Interactive Software, Inc.
    'LNTH',  # Lantheus Holdings, Inc.
    'FICO',  # Fair Isaac Corporation (FICO)
]

# Chemicals and Energy
tickers += [
    'CL',    # Continental Resources, Inc.
    'NEE',   # NextEra Energy, Inc.
    'CCJ',   # Cameco Corporation
    'GEVO',  # Gevo, Inc.
]

# Check which tickers in your tickers list are not in all_tickers
missing_tickers = [ticker for ticker in tickers if ticker not in all_tickers]

# Print results
if not missing_tickers:
    print("All tickers are included in all_tickers.")
else:
    print("Missing tickers:", missing_tickers)
    
all_tickers += missing_tickers
all_tickers.sort()

# tickers = all_tickers

/Users/richazhu/Desktop/MyValues
Missing tickers: ['^IXIC', '^VIX', '^TNX', '^IRX', 'TLT', 'TMF', 'SHY', 'EDV', 'ZROZ', 'GOVT', 'VXX', 'UVXY', 'MTUM', 'SPY', 'QQQ', 'QQQM', 'VOOG', 'VGT', 'XLU', 'GLD', 'IAU', 'PHYS', 'GDX', 'SLV', 'IAU']


# Technical Indicators

## Custom Indicators - High Momentum Factor / High Quality Score (HMHQ)

In [4]:
%%capture

# Initialize a dictionary to store results for each ticker
results = {}

# Define a target return (for instance, the risk-free rate)
target_return = 0.0  # Adjust this as needed
rolling_window = 20   # Define the window size for rolling calculations

# Define the benchmark
benchmark_ticker = 'SPY'

# ** Data Processing Loop for Each Ticker **
for ticker in tickers:
    # ** 0A. Data Downloading Section **
    try:
        data = yf.download(ticker, start=(datetime.now() - timedelta(days=2*365)).strftime("%Y-%m-%d"), end=datetime.now().strftime("%Y-%m-%d"), interval="1d")

        # Check if the data is empty
        if data.empty:
            print(f"No data found for {ticker}. Skipping.")
            continue
        
        # Add benchmark data for Beta calculation
        if ticker == benchmark_ticker:
            benchmark_data = data['Close']
            continue  # Skip the rest for the benchmark itself

        # ** 0. Moving Averages Section **
        data['3_SMA'] = data['Close'].rolling(window=3).mean()
        data['5_SMA'] = data['Close'].rolling(window=5).mean()
        data['20_SMA'] = data['Close'].rolling(window=20).mean()
        data['50_SMA'] = data['Close'].rolling(window=50).mean()
        data['90_SMA'] = data['Close'].rolling(window=90).mean()
        data['180_SMA'] = data['Close'].rolling(window=180).mean()
        data['360_SMA'] = data['Close'].rolling(window=360).mean()
   
        # ** 0. Price Change Calculations Section **
        data['Price_Change%'] = data['Close'].pct_change() * 100 
        data['Cumulative_Returns'] = (1 + data['Price_Change%'] / 100).cumprod()
             
        # ** 1. Market Trend Section - when 3_SMA > 5_SMA **
        data['Market_Trend'] = 0
        data.loc[data['3_SMA'] > data['5_SMA'], 'Market_Trend'] = 1
        data.loc[data['3_SMA'] < data['5_SMA'], 'Market_Trend'] = -1

        # ** 2. Rolling Maximum Drawdown Calculation Section **
        data['Rolling_Peak'] = data['Cumulative_Returns'].cummax()
        data['Rolling_Drawdown'] = (data['Cumulative_Returns'] - data['Rolling_Peak']) / data['Rolling_Peak']

        # ** 3. Cumulative Change Calculation Section using vectorized operations **
        current_trend_shifted = data['Market_Trend'].shift(1)
        current_base_price = data['Close'].shift(1).where(current_trend_shifted != data['Market_Trend']).ffill()
        data['Cumulative_Change%'] = ((data['Close'] - current_base_price) / current_base_price) * 100
        data['Over_10%_in_90_days'] = data['Cumulative_Change%'].rolling(window=90, min_periods=1).max() > 10
        data['Over_10%_in_90_days'] = data['Over_10%_in_90_days'].astype(int)

        # ** 4. Rolling Up/Down Ratio Calculation **
        data['Up_Day'] = data['Price_Change%'].apply(lambda x: x if x > 0 else 0)
        data['Down_Day'] = data['Price_Change%'].apply(lambda x: -x if x < 0 else 0)
        data['Rolling_Up_Down_Ratio'] = data['Up_Day'].rolling(window=rolling_window).mean() / data['Down_Day'].rolling(window=rolling_window).mean()
        
        # ** 5. Rolling True Strength Index (TSI) Calculation **
        data['Smooth_Price_Change'] = data['Price_Change%'].ewm(span=25, adjust=False).mean()  # Fast TSI
        data['Smooth_Price_Change_Slow'] = data['Price_Change%'].ewm(span=13, adjust=False).mean()  # Slow TSI
        data['TSI'] = 100 * (data['Smooth_Price_Change'] - data['Smooth_Price_Change_Slow']) / data['Smooth_Price_Change_Slow']
        data['Rolling_TSI'] = data['TSI'].rolling(window=rolling_window).mean()

        # ** 6. Rolling Volatility Calculation **
        data['Rolling_Volatility'] = data['Price_Change%'].rolling(window=rolling_window).std()

        # ** 7. Rolling Maximum Consecutive Up Days Calculation **
        data['Consecutive_Up_Days'] = (data['Price_Change%'] > 0).astype(int)
        data['Rolling_Consecutive_Up_Days'] = data['Consecutive_Up_Days'].groupby((data['Consecutive_Up_Days'] != data['Consecutive_Up_Days'].shift()).cumsum()).cumsum()
        
        # ** 8. Max Rolling Consecutive Up Days in a 20-day Window **
        consecutive_counts = data['Consecutive_Up_Days'].rolling(window=rolling_window).apply(
            lambda x: (x != 0).astype(int).groupby((x != 0).ne((x.shift())).cumsum()).cumsum().max(),
            raw=False
        )
        data['Max_Rolling_Consecutive_Up_Days'] = consecutive_counts
        
        consecutive_counts = data['Consecutive_Up_Days'].rolling(window=90).apply(
            lambda x: (x != 0).astype(int).groupby((x != 0).ne((x.shift())).cumsum()).cumsum().max(),
            raw=False
        )
        data['Max_SemiAnnual_Consecutive_Up_Days'] = consecutive_counts
        data['Potential_Up_Days'] = data['Max_SemiAnnual_Consecutive_Up_Days']-data['Consecutive_Up_Days']

        # ** 9. RSI Calculation (Relative Strength Index) **
        delta = data['Close'].diff(1)
        gain = (delta.where(delta > 0, 0)).rolling(window=rolling_window).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=rolling_window).mean()
        rs = gain / loss
        data['RSI'] = 100 - (100 / (1 + rs))

        # ** 10. Momentum Calculation **
        data['Momentum'] = data['Close'].diff(rolling_window)

        # ** 11. ATR Calculation (Average True Range) **
        data['High_Low'] = data['High'] - data['Low']
        data['High_Close'] = (data['High'] - data['Close'].shift(1)).abs()
        data['Low_Close'] = (data['Low'] - data['Close'].shift(1)).abs()
        data['True_Range'] = data[['High_Low', 'High_Close', 'Low_Close']].max(axis=1)
        data['ATR'] = data['True_Range'].rolling(window=rolling_window).mean()

        # ** 12. Beta Calculation **
        if 'benchmark_data' in locals():
            returns = data['Close'].pct_change()
            benchmark_returns = benchmark_data.pct_change()
            covariance = returns.rolling(window=rolling_window).cov(benchmark_returns)
            variance = benchmark_returns.rolling(window=rolling_window).var()

            data['Beta'] = covariance / variance

        # ** 13. Alpha Calculation **
        if 'benchmark_data' in locals():
            average_benchmark_return = benchmark_data.pct_change().rolling(window=rolling_window).mean()
            data['Alpha'] = (data['Price_Change%'].rolling(window=rolling_window).mean() - (data['Beta'] * average_benchmark_return)).fillna(0)

        # ** 14. Preparing the Trend Indicator DataFrame **
        trend_indicator = data[['Close', 
                                'Price_Change%', 
                                'Market_Trend', 
                                'Cumulative_Change%', 
                                'Over_10%_in_90_days',
                                'Rolling_Drawdown', 
                                'Rolling_Up_Down_Ratio', 
                                'Rolling_TSI', 
                                'Rolling_Volatility', 
                                'Rolling_Consecutive_Up_Days', 
                                'Max_Rolling_Consecutive_Up_Days', 
                                'Max_SemiAnnual_Consecutive_Up_Days',
                                'Potential_Up_Days',
                                'RSI', 
                                'Momentum',
                                'ATR']].dropna()

        # Conditionally adding Alpha and Beta if they exist in the DataFrame
        trend_columns = [
            f'{ticker} Close',
            f'{ticker} daily move%',
            f'{ticker} ST trend signal',
            f'{ticker} cumulative change since signal',
            f'{ticker} cumulative change 90-day breakthrough',
            f'{ticker} Rolling Max Drawdown',
            f'{ticker} Rolling Up Down Ratio',
            f'{ticker} Rolling TSI',
            f'{ticker} Rolling Volatility',
            f'{ticker} Consecutive Up Days',
            f'{ticker} Max Rolling Consecutive Up Days',
            f'{ticker} Max_SemiAnnual_Consecutive_Up_Days',
            f'{ticker} Potential_Up_Days',
            f'{ticker} RSI',
            f'{ticker} Momentum',
            f'{ticker} ATR'
        ]

        # Check if Alpha and Beta exist in data for column addition
        if 'Alpha' in data.columns:
            trend_indicator[f'{ticker} Alpha'] = data['Alpha']
            trend_columns.append(f'{ticker} Alpha')

        if 'Beta' in data.columns:
            trend_indicator[f'{ticker} Beta'] = data['Beta']
            trend_columns.append(f'{ticker} Beta')

        trend_indicator.columns = trend_columns
        
        # ** 15. Sortino Ratio Calculation Section **
        average_return = data['Price_Change%'].rolling(window=rolling_window).mean()
        downside_returns = data['Price_Change%'][data['Price_Change%'] < target_return].rolling(window=rolling_window)
        downside_deviation = downside_returns.apply(lambda x: (x ** 2).mean() ** 0.5 if not x.empty else 0)
        data['Sortino Ratio'] = (average_return - target_return) / downside_deviation
        data['Sortino Ratio'].replace([float('inf'), -float('inf')], 0, inplace=True)  # Replace infinities

        # ** 16. Calmar Ratio Calculation Section **
        annualized_return = ((1 + average_return / 100) ** 252) - 1
        data['Calmar Ratio'] = annualized_return / abs(data['Rolling_Drawdown'].min())  # Use current rolling drawdown

        # ** 17. Sharpe Ratio Calculation Section **
        trend_indicator[f'{ticker} Sortino Ratio'] = data['Sortino Ratio']
        trend_indicator[f'{ticker} Calmar Ratio'] = data['Calmar Ratio']

        results[ticker] = trend_indicator  # Store the results for this ticker

    except Exception as e:
        print(f"Error processing {ticker}: {e}")

# ** Risk-Free Rate Calculation Section **
if "^TNX" in results:  # Ensure ^TNX has been processed successfully
    treasury_yield = results["^TNX"]['^TNX Close']
    risk_free_rate = treasury_yield / 100 / 252  # Convert from annual to daily
else:
    print("Warning: ^TNX results not found. Defaulting risk-free rate to 0.")
    risk_free_rate = 0.0  # Fallback risk-free rate

# ** Calculate Sharpe Ratio for all tickers using the risk-free rate **
for ticker in tickers:
    if ticker in results:
        data = results[ticker]
        rolling_average_return = data[f'{ticker} daily move%'].rolling(window=rolling_window).mean()
        rolling_std_dev_returns = data[f'{ticker} daily move%'].rolling(window=rolling_window).std()

        # Define Sharpe Ratio; replace infinities for stability
        data[f'{ticker} Sharpe Ratio'] = (rolling_average_return - risk_free_rate) / rolling_std_dev_returns
        data[f'{ticker} Sharpe Ratio'].replace([float('inf'), -float('inf')], 0, inplace=True)  # Replace infinities
        results[ticker] = data  # Update results with Sharpe Ratio

# ** Display Results Section **
# for ticker in tickers:
#     if ticker in results:  # Only display results for tickers that were processed successfully
#         print(f"Results for {ticker}:")
#         display(results[ticker].tail(2))  # Display the last 10 entries

# Get the indices where the daily move percentage is less than -2%
high_move_indices = results['^IXIC'][results['^IXIC']['^IXIC daily move%'] < -2].index

# Convert DatetimeIndex to a list of datetime objects
date_list = high_move_indices.tolist()

# Convert the dates to strings formatted as 'YYYY-MM-DD'
date_list_as_strings = [date.strftime('%Y-%m-%d') for date in date_list]

# Display the formatted list of dates
print("List of dates with a daily move less than -2%:", date_list_as_strings)

# Iterate over each ticker in the results
for ticker in results:
    # Access the DataFrame for the current ticker
    df = results[ticker]
    
    # Create a new column for the reversed_trend_flag, initially set to 0
    df[f'{ticker} anti-drawdown flag'] = 0
    
    # Convert date_list_as_strings to datetime objects for comparison
    date_list_as_datetimes = pd.to_datetime(date_list_as_strings)
    
    # Create a mask that checks for dates in date_list_as_datetimes
    mask = df.index.isin(date_list_as_datetimes)
    
    # Set the reversed_trend_flag to 1 for those dates if daily move% > -0.5%
    df.loc[mask, f'{ticker} anti-drawdown flag'] = (df.loc[mask, f'{ticker} daily move%'] > -0.5).astype(int)

    # Update the results dictionary with the modified DataFrame
    results[ticker] = df
    
    # mask = results['SFM'].index.isin(date_list_as_datetimes)
    # filtered_df = results['SFM'][mask]
    # filtered_df
    
    # Set your rolling window size
    rolling_window_size = 180  # Adjust this as needed

    # Calculate the rolling average of 'anti-drawdown flag' and create a new column
    results[ticker][f'{ticker} Rolling Average Anti Drawdown'] = results[ticker][f'{ticker} anti-drawdown flag'].rolling(window=rolling_window_size).sum()


In [5]:
subdirectory = 'data_outputs'  # This is your target subfolder
file_name = 'technical_indicators_HMHQ.xlsx'
file_path = os.path.join(current_directory, subdirectory, file_name)  # Full path for the file

# Create a Pandas Excel writer using XlsxWriter as the engine.
with pd.ExcelWriter(file_path, engine='xlsxwriter') as writer:
    for ticker, df in results.items():
        df.to_excel(writer, sheet_name=ticker)  # Save each ticker's DataFrame to a separate sheet

In [6]:
# Initialize an empty list to store modified DataFrames
modified_dfs = []

# Iterate over all tickers and their corresponding DataFrames
for ticker, df in results.items():
    # Reset the index to make the date a column
    df_reset = df.reset_index(drop=False)  # drop=False keeps the index

    # Add a new column for the ticker
    df_reset['Ticker'] = ticker

    # Rename columns to remove the ticker string
    # Assuming columns with ticker are structured like '{ticker} ColumnName'
    df_reset.columns = [col.replace(f'{ticker} ', '') for col in df_reset.columns]
    
    # Drop the 'Trend signal' column if it exists    
    if 'Strend signal' in df_reset.columns:        
        df_reset.drop(columns=['Strend signal'], inplace=True)
    
    # Append the modified DataFrame to the list
    modified_dfs.append(df_reset)

# Combine all modified DataFrames into a single DataFrame
combined_df = pd.concat(modified_dfs, ignore_index=True)

subdirectory = 'data_outputs'  # This is your target subfolder
file_name = 'technical_indicators_HMHQ.csv'
file_path = os.path.join(current_directory, subdirectory, file_name)  # Full path for the file

# Now you can save the combined DataFrame to a CSV file
combined_df.to_csv(file_path, index=False)

# Print out the first few rows of the combined DataFrame
display(combined_df.head())

,Date,Close,daily move%,ST trend signal,cumulative change since signal,cumulative change 90-day breakthrough,Rolling Max Drawdown,Rolling Up Down Ratio,Rolling TSI,Rolling Volatility,...,Momentum,ATR,Alpha,Beta,Sortino Ratio,Calmar Ratio,Sharpe Ratio,anti-drawdown flag,Rolling Average Anti Drawdown,Ticker
0,2023-03-22,11669.959961,-1.603277,1.0,2.115912,0,-0.043510,1.158259,-17.937931,1.342291,...,162.889648,226.306982,0.079654,1.075255,0.058187,1.671294,NaN,0,NaN,^IXIC
1,2023-03-23,11787.400391,1.006348,1.0,3.143553,0,-0.033885,1.186577,-14.299621,1.350886,...,197.000000,230.623486,0.093911,1.082886,NaN,2.006795,NaN,0,NaN,^IXIC
2,2023-03-24,11823.959961,0.310158,-1.0,0.310158,0,-0.030888,1.465754,-12.516880,1.284620,...,429.019531,225.604980,0.192804,1.038756,NaN,4.752010,NaN,0,NaN,^IXIC
3,2023-03-27,11768.839844,-0.466173,1.0,-0.466173,0,-0.035406,1.315321,-9.823225,1.288319,...,301.859375,225.298486,0.137964,1.031187,0.101535,3.157224,NaN,0,NaN,^IXIC
4,2023-03-28,11716.080078,-0.448300,1.0,-0.912384,0,-0.039730,1.264906,-5.587731,1.294047,...,260.540039,226.346924,0.120463,1.036212,0.088501,2.695363,NaN,0,NaN,^IXIC


In [7]:
%%capture

# Initialize a dictionary to hold the correlations with target stock
correlation_results = {}

# List of relevant indicators for correlation analysis
indicators = [
    'ST trend signal',
    'cumulative change since signal',
    'cumulative change 90-day breakthrough',
    'Rolling Max Drawdown',
    'Rolling Average Anti Drawdown',
    'Rolling Up Down Ratio',
    'Rolling TSI',
    'Rolling Volatility',
    'Potential_Up_Days',
    'Consecutive Up Days',
    'Max Rolling Consecutive Up Days',
    'RSI',
    'Momentum',
    'ATR',
    'Alpha',
    'Beta',
    'Sortino Ratio',
    'Calmar Ratio',
    'Sharpe Ratio'
]

# Create a list of DataFrames for other tickers
other_tickers_dfs = results

# Calculate correlations for each indicator with AAPL
for ticker, df_other in other_tickers_dfs.items():
    correlations = {}
    for indicator in indicators:
        focus_col = f'SFM {indicator}'
        other_col = f'{ticker} {indicator}'

        # Ensure both columns exist before calculating
        if focus_col in other_tickers_dfs['SFM'].columns and other_col in df_other.columns:
            try:
                correlation = other_tickers_dfs['SFM'][focus_col].corr(df_other[other_col])
                correlations[indicator] = correlation
            except Exception as e:
                print(f"Error calculating correlation for {ticker}: {str(e)}")

    # Calculate the ensemble correlation for the current ticker
    if correlations:  # Check if any correlations were calculated
        ensemble_correlation = np.mean(list(correlations.values()))
        correlation_results[ticker] = ensemble_correlation

# Convert the results dictionary into a DataFrame for better readability
results_df = pd.DataFrame(correlation_results.items(), columns=['Ticker', 'Ensemble Correlation'])

# Sort results_df by 'Ensemble Correlation' in descending order
sorted_results_df = results_df.sort_values(by='Ensemble Correlation', ascending=False)

In [8]:
subdirectory = 'data_outputs'  # This is your target subfolder
file_name = 'HMHQ_score_ranking.csv'
file_path = os.path.join(current_directory, subdirectory, file_name)  # Full path for the file

sorted_normalized_results_df = pd.read_csv(file_path)
display(sorted_normalized_results_df)

,Reporting_Date,Ticker,Normalized Weighted Score,Simple Average Score,Ranking
0,2024-11-08,SFM,100.000000,1.000000,1.0
1,2024-11-08,SMR,34.520711,0.207802,2.0
2,2024-11-08,IDCC,31.580485,0.215100,3.0
3,2024-11-08,PHYS,30.689850,0.179418,4.0
4,2024-11-08,CL,29.352112,0.159810,5.0
...,...,...,...,...,...
62,2024-11-08,AFRM,NaN,-0.016196,NaN
63,2024-11-08,ESOA,NaN,0.064622,NaN
64,2024-11-08,VNO,NaN,0.081316,NaN
65,2024-11-08,RVMD,NaN,0.140718,NaN


In [9]:
# Initialize a dictionary to hold the correlations with the target stock
correlation_results = {}

# List of relevant indicators for correlation analysis
indicators = [
    'ST trend signal',
    'cumulative change since signal',
    'cumulative change 90-day breakthrough',
    'Rolling Max Drawdown',
    'Rolling Average Anti Drawdown',
    'Rolling Up Down Ratio',
    'Rolling TSI',
    'Rolling Volatility',
    'Potential_Up_Days',
    'Consecutive Up Days',
    'Max Rolling Consecutive Up Days',
    'RSI',
    'Momentum',
    'ATR',
    'Alpha',
    'Beta',
    'Sortino Ratio',
    'Calmar Ratio',
    'Sharpe Ratio'
]

# Define relative weights for each indicator
weights = [
    5,    # Weight for 'ST trend signal'
    15,   # Weight for 'cumulative change since signal'
    15,    # Weight for 'cumulative change 90-day breakthrough'
    15,   # Weight for 'Rolling Max Drawdown'
    15,   # Weight for 'Rolling Average Anti Drawdown'
    20,   # Weight for 'Rolling Up Down Ratio'
    10,   # Weight for 'Rolling TSI'
    5,    # Weight for 'Rolling Volatility'
    15,   # Weight for 'Potential_Up_Days'
    5,    # Weight for 'Consecutive Up Days'
    10,   # Weight for 'Max Rolling Consecutive Up Days'
    20,   # Weight for 'RSI'
    15,   # Weight for 'Momentum'
    5,    # Weight for 'ATR'
    5,    # Weight for 'Alpha'
    5,    # Weight for 'Beta'
    15,   # Weight for 'Sortino Ratio'
    15,   # Weight for 'Calmar Ratio'
    15    # Weight for 'Sharpe Ratio'
]

# Create a list of DataFrames for other tickers
other_tickers_dfs = results

# Calculate correlations for each indicator with SFM
for ticker, df_other in other_tickers_dfs.items():
    correlations = {}
    for indicator in indicators:
        focus_col = f'SFM {indicator}'
        other_col = f'{ticker} {indicator}'

        # Ensure both columns exist before calculating
        if focus_col in other_tickers_dfs['SFM'].columns and other_col in df_other.columns:
            try:
                correlation = other_tickers_dfs['SFM'][focus_col].corr(df_other[other_col])
                correlations[indicator] = correlation
            except Exception as e:
                print(f"Error calculating correlation for {ticker} using {indicator}: {str(e)}")

    # Calculate the weighted ensemble correlation for the current ticker
    if correlations:  # Check if any correlations were calculated
        weighted_sum = sum(correlations[indicator] * weights[i] for i, indicator in enumerate(indicators) if indicator in correlations)
        correlation_results[ticker] = {"weighted_sum": weighted_sum, "correlations": correlations}
    else:
        print(f"No valid correlations calculated for {ticker}")

# Debug: Print correlation_results to check for NaN values
# print("Correlation Results:", correlation_results)

# After all results are calculated, determine the min and max of the weighted sums
if correlation_results:
    valid_sums = [v["weighted_sum"] for v in correlation_results.values() if pd.notna(v["weighted_sum"])]

    min_weighted_sum = min(valid_sums) if valid_sums else 0
    max_weighted_sum = max(valid_sums) if valid_sums else 100  # To avoid max equal to min

    # Normalizing the weighted sums to a range of 0 to 100
    normalized_results = {}
    simple_averages = {}

    for ticker, result in correlation_results.items():
        weighted_sum = result["weighted_sum"]
        correlations = result["correlations"]

        # Calculate simple average only of valid correlations
        valid_correlations = [v for v in correlations.values() if pd.notna(v)]
        simple_average = sum(valid_correlations) / len(valid_correlations) if valid_correlations else 0
        
        # Normalize the weighted sum
        normalized_score = ((weighted_sum - min_weighted_sum) / (max_weighted_sum - min_weighted_sum) * 100) if max_weighted_sum > min_weighted_sum else 0

        normalized_results[ticker] = normalized_score
        simple_averages[ticker] = simple_average

# Convert the results into a DataFrame
results_df = pd.DataFrame({
    'Reporting_Date': datetime.now().strftime("%Y-%m-%d"),
    'Ticker': normalized_results.keys(),
    'Normalized Weighted Score': normalized_results.values(),
    'Simple Average Score': simple_averages.values()
})

# Sort results_df by 'Normalized Weighted Score' in descending order
sorted_normalized_results_df = results_df.sort_values(by='Normalized Weighted Score', ascending=False)
sorted_normalized_results_df['Ranking'] = sorted_normalized_results_df['Normalized Weighted Score'].rank(method='min', ascending=False)

# Display the sorted results
print("Sorted Normalized Weighted Scores with Simple Averages:")
display(sorted_normalized_results_df[:5])

sorted_normalized_results_df.to_csv(file_path, mode = 'w', header=True, index=False)

Sorted Normalized Weighted Scores with Simple Averages:


/opt/anaconda3/lib/python3.11/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/opt/anaconda3/lib/python3.11/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


,Reporting_Date,Ticker,Normalized Weighted Score,Simple Average Score,Ranking
33,2024-11-09,SFM,100.000000,1.000000,1.0
49,2024-11-09,SMR,32.028526,0.215882,2.0
50,2024-11-09,IDCC,28.837406,0.219950,3.0
30,2024-11-09,PHYS,27.096233,0.178551,4.0
27,2024-11-09,COST,25.854927,0.165329,5.0
